
#  How compilers translate your program


Enter your name and student ID.

 * Name:
 * Student ID:



<a name="intro"> </a>
# 1. Introduction
* You look at assembly code generated by the compiler to get a good sense of how source programs are compiled into machine (assembly) code



# 2. Let the compiler generate assembly code
* you first have to master how to let your compiler emit assembly code 
* this is language- or compiler-dependent


## 2-1. Go
* ordinary go compiler does not support emitting native 64 bit x86 assembly
* it instead emits something called plan 9 assembly ([details](https://go.dev/doc/asm)) which abstracts away important details
* in this exercise, we instead use an alternative compiler, `gccgo`, which generates a native assembly for x86
* here is a minimum example

* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile go/add123.go`) into the specified file `go/add123.go`
* note that it is Python kernel and you don't have to (and should not) change it to `go` kernel

In [ ]:
%%writefile go/add123.go

package pl06
func Add123(n int64) int64 {
    return n + 123
}

* `-S` to emit assembly code
* `-O` to optimize
* `-g0` not to generate debugging information (unncessary for now)

In [ ]:
gccgo -O -g0 -S go/add123.go -o go/add123.s
cat go/add123.s


* find the label starting the function `Add123`, which may not be exactly `Add123`

* **NOTE 1:**
  * you need to say `package xxx`, with _xxx_ anything other than `main`; if you say `package main`, the compiler complains about the lack of `main` function
  * you need to <font color="red">capitalize the function name</font> whose assembly code you want to see, as otherwise the compiler may eliminate it as a dead code
  * in Go, functions starting with an uppercase letter are visible (can be called) from another package (see https://go.dev/doc/tutorial/create-module), so the compiler should leave the assembly code even if it finds the function not called anywhere in the file it is compiling

* **NOTE 2:** if you want to use `gccgo` along with `go build`, you can do
```
go build -compiler gccgo 
```
instead of just 
```
go build
```

## 2-2. Julia
* Julia has <font color="blue">code_native</font> function that prints assembly code of a function given particular concrete types for its input parameters
* here is a minimum example

* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile jl/add123.jl`) into the specified file `jl/add123.jl`
* note that it is Python kernel and you don't have to (and should not) change it to `Julia` kernel


In [ ]:
%%writefile jl/add123.jl

import InteractiveUtils

function add123(n)
    n + 123
end

InteractiveUtils.code_native(add123, (Int,))

* `code_native` shows assembly code for a specified function

In [ ]:
julia jl/add123.jl

* find the label starting the function `add123`, which may not be exactly `add123`

* note that the function `add123` does not declare the type of the input parameter (`n`); instead, the second parameter of `code_native`, `(Int,)` instructs Julia to compile `add123` given the type of the input is `Int`
* you can see how the same function is compiled for floating point numbers just by `code_native(add123, (Float64,))`


## 2-3. OCaml
* `ocamlopt` is the native compiler for OCaml, which is internally called by `dune`
* here is a minimum example

* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile ml/add123.ml`) into the specified file `ml/add123.ml`
* note that it is Python kernel and you don't have to (and should not) change it to `OCaml` kernel

In [ ]:
%%writefile ml/add123.ml

let add123 n = n + 123
;;

* `ocamlopt` is a "native compiler" that translates OCaml source into machine code (`ocamlc` compiles OCaml to bytecode)
* `-S` to emit assembly code


In [ ]:
ocamlopt -S ml/add123.ml
cat ml/add123.s

* find the label starting the function `add123`, which may not be exactly `add123`

## 2-4. Rust
* `rustc` is a compiler for Rust, which is internally called by `cargo`
* here is a minimum example

* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile rs/add123.rs`) into the specified file `rs/add123.rs`
* note that it is Python kernel and you don't have to (and should not) change it to `Rust` kernel


In [ ]:
%%writefile rs/add123.rs

#[no_mangle]
pub fn add123(n : i64) -> i64 {
    n + 123
}

* `-O` to optimize
* `--emit asm` to generate assembly code
* `--crate-type lib` to say this is for a library, not an executable

In [ ]:
rustc -O --emit asm --crate-type lib rs/add123.rs -o rs/add123.s
cat rs/add123.s

* try to find out where the function starts and instructions actually performing the addition

* **NOTE:**
  * `--crate-type lib` says you are building a library which will be called from another program, not an executable; without it, the compiler complains about the lack of `main` function
  * `#[no_mangle]` guarantees the label name corresponding to `add123` is exactly `add123` and has a side-effect of preventing the compiler from eliminating the function as dead code (I try to find a way to guarantee `add123` is not eliminatd without `#[no_mangle]`, but couldn't.  let me know if you know a better way)


# <font color="green"> Problem 1 :  Calling convention (where are args and return values?)</font>
* define a function `many_args` (or `Many_args` in Go) that takes many (> 10) integer parameters and returns the sum of them (similar to the following C function) in your language, compile it into assembly and examine it

In [ ]:
%%writefile cc/many_args.c

long many_args(long a00, long a01, long a02, long a03, long a04, long a05,
               long a06, long a07, long a08, long a09, long a10, long a11) {
  return a00 + a01 + a02 + a03 + a04 + a05 + a06 + a07 + a08 + a09 + a10 + a11;
}

In [ ]:
gcc -O3 -S cc/many_args.c -o cc/many_args.s
cat cc/many_args.s

* write the code below
* for Julia, do not forget to include `import InteractiveUtils` and `InteractiveUtils.code_native(YOUR_FUNCTION, (TYPE,...))` to show assembly code

In [ ]:
# replace xx with go, jl, ml, or rs and write the function
%%writefile xx/many_args.c


* and see the assembly code 

In [ ]:
# uncomment commands for your language
# Go
# gccgo -O -g0 -S go/many_args.go -o go/many_args.s
# cat go/many_args.go

# Julia
# julia jl/many_args.jl

# OCaml
# ocamlopt -S ml/many_args.ml
# cat ml/many_args.ml

# Rust
# rustc -O --emit asm --crate-type lib rs/many_args.rs -o rs/many_args.s
# cat rs/many_args.s

* __<font color="blue">Questions:</font>__
  * how are parameters passed in each language? (which registers? stack?)
  * how does your language represent an integer $x$? is it represented in the same way as C?
  * does your language's calling convention differ from the C calling convention?



# <font color="green"> Problem 2 :  How floating point numbers are represented</font>
* define a function `add_floats` (or `Add_floats` in Go) that takes two floating point numbers and returns the sum of the two (similar to the following C function) in your language, compile it into assembly and examine it

In [ ]:
%%writefile cc/add_floats.c

double add_floats(double x, double y) {
  return x + y;
}

In [ ]:
gcc -O3 -S cc/add_floats.c -o cc/add_floats.s
cat cc/add_floats.s

* write the code below
* for Julia, do not forget to include `import InteractiveUtils` and `InteractiveUtils.code_native(YOUR_FUNCTION, (TYPE,...))` to show assembly code

In [ ]:
# replace xx with go, jl, ml, or rs and write the function
%%writefile xx/add_floats.c


* and see the assembly code 

In [ ]:
# uncomment commands for your language
# Go
# gccgo -O -g0 -S go/add_floats.go -o go/add_floats.s
# cat go/add_floats.go

# Julia
# julia jl/add_floats.jl

# OCaml
# ocamlopt -S ml/add_floats.ml
# cat ml/add_floats.ml

# Rust
# rustc -O --emit asm --crate-type lib rs/add_floats.rs -o rs/add_floats.s
# cat rs/add_floats.s

* __<font color="blue">Questions:</font>__
  * how are the two parameters passed in each language? (which registers? stack?)
  * how does your language represent a floating point number $x$? is it represented in the same way as C?
  * is there any language whose floating point number computations will be significantly slower than C?



# <font color="green"> Problem 3 :  How arrays are represented</font>
* define a function `get_float_array_elem` (or `get_float_array_elem` in Go) that takes an array of floating point numbers, `a`, and an integer, `i`, and returns `i`-th element of `a` (similar to to the following C function) in your language, compile it into assembly and examine it
* for an array, use
  * `[]float64` (slice) for Go
  * `Vector{Float64}` for Julia
  * `float array` (slice) for Go
  * `Vec<f64>` for Rust

In [ ]:
%%writefile cc/get_float_array_elem.c

double get_float_array_elem_const(double a[10]) {
  return a[2];
}
double get_float_array_elem_i(double a[10], long i) {
  return a[i];
}

In [ ]:
gcc -O3 -S cc/get_float_array_elem.c -o cc/get_float_array_elem.s
cat cc/get_float_array_elem.s

* write the code below
* for Julia, do not forget to include `import InteractiveUtils` and `InteractiveUtils.code_native(YOUR_FUNCTION, (TYPE,...))` to show assembly code

In [ ]:
# replace xx with go, jl, ml, or rs and write the function
%%writefile xx/get_float_array_elem.c


* and see the assembly code 

In [ ]:
# uncomment commands for your language
# Go
# gccgo -O -g0 -S go/get_float_array_elem.go -o go/get_float_array_elem.s
# cat go/get_float_array_elem.go

# Julia
# julia jl/get_float_array_elem.jl

# OCaml
# ocamlopt -S ml/get_float_array_elem.ml
# cat ml/get_float_array_elem.ml

# Rust
# rustc -O --emit asm --crate-type lib rs/get_float_array_elem.rs -o rs/get_float_array_elem.s
# cat rs/get_float_array_elem.s

* __<font color="blue">Questions:</font>__
  * how does each language represent the respective array-like data structure?
  * how does each language detect out-of-bound array access? 



# <font color="green"> Problem 4 :  How structs are represented</font>
* define a struct `point` (or `Point` in Go) that has two fields (`x` and `y`; both are 64 bit floating point numbers) and function `get_struct_elem` (or `Get_struct_elem` in Go) that takes a point or a pointer to it and returns its `y` field

In [ ]:
%%writefile cc/get_struct_elem.c

typedef struct {
  double x;
  double y;
} point_t;  
double get_point_y(point_t p) {
  return p.y;
}
double get_pointp_y(point_t * p) {
  return p->y;
}

In [ ]:
gcc -O3 -S cc/get_struct_elem.c -o cc/get_struct_elem.s
cat cc/get_struct_elem.s

* write the code below
* for Julia, do not forget to include `import InteractiveUtils` and `InteractiveUtils.code_native(YOUR_FUNCTION, (TYPE,...))` to show assembly code

In [ ]:
# replace xx with go, jl, ml, or rs and write the function
%%writefile xx/get_struct_elem.c


* and see the assembly code 

In [ ]:
# uncomment commands for your language
# Go
# gccgo -O -g0 -S go/get_struct_elem.go -o go/get_struct_elem.s
# cat go/get_struct_elem.go

# Julia
# julia jl/get_struct_elem.jl

# OCaml
# ocamlopt -S ml/get_struct_elem.ml
# cat ml/get_struct_elem.ml

# Rust
# rustc -O --emit asm --crate-type lib rs/get_struct_elem.rs -o rs/get_struct_elem.s
# cat rs/get_struct_elem.s

* __<font color="blue">Questions:</font>__
  * how does each language represent multiword structures?
  * does each language possibly get a null pointer?
  * if so, how it preparse for it?



# <font color="green"> Problem 5 :  If statement/expressions</font>
* define a function `collatz` (or `Collatz` in Go) that takes an integer $n$ and returns $n/2$ if $n$ is an even number and $3 n + 1$ otherwise (similar to the following C function)

In [ ]:
%%writefile cc/collatz.c

long collatz(long n) {
  if (n % 2 == 0) {
    return n / 2;
  } else {
    return 3 * n + 1;
  }
}

In [ ]:
gcc -O3 -S cc/collatz.c -o cc/collatz.s
cat cc/collatz.s

* write the code below
* for Julia, do not forget to include `import InteractiveUtils` and `InteractiveUtils.code_native(YOUR_FUNCTION, (TYPE,...))` to show assembly code

In [ ]:
# replace xx with go, jl, ml, or rs and write the function
%%writefile xx/collatz.c


* and see the assembly code 

In [ ]:
# uncomment commands for your language
# Go
# gccgo -O -g0 -S go/collatz.go -o go/collatz.s
# cat go/collatz.go

# Julia
# julia jl/collatz.jl

# OCaml
# ocamlopt -S ml/collatz.ml
# cat ml/collatz.ml

# Rust
# rustc -O --emit asm --crate-type lib rs/collatz.rs -o rs/collatz.s
# cat rs/collatz.s


# <font color="green"> Problem 6 :  Function call</font>
* define a function `linear_rec` (or `Linear_rec` in Go) that takes an integer $n$ and returns $a_n$ of the following recurrence

$$ \begin{array}{rcl} a_0 & = & 1, \\ a_n & = & (123 a_{n-1} + 45) \mod 67 \quad (n > 0) \end{array} $$

* write it in the straightforward recursion as you have done in the functional programming exercise  (similar to the following Cfunction)

In [ ]:
%%writefile cc/linear_rec.c

long linear_rec(long n) {
  if (n == 0) {
    return 12;
  } else {
    return (34 * linear_rec(n - 1) + 56) % 78;
  }
}

In [ ]:
gcc -O3 -S cc/linear_rec.c -o cc/linear_rec.s
cat cc/linear_rec.s

* write the code below
* for Julia, do not forget to include `import InteractiveUtils` and `InteractiveUtils.code_native(YOUR_FUNCTION, (TYPE,...))` to show assembly code

In [ ]:
# replace xx with go, jl, ml, or rs and write the function
%%writefile xx/linear_rec.c


* and see the assembly code 

In [ ]:
# uncomment commands for your language
# Go
# gccgo -O -g0 -S go/linear_rec.go -o go/linear_rec.s
# cat go/linear_rec.go

# Julia
# julia jl/linear_rec.jl

# OCaml
# ocamlopt -S ml/linear_rec.ml
# cat ml/linear_rec.ml

# Rust
# rustc -O --emit asm --crate-type lib rs/linear_rec.rs -o rs/linear_rec.s
# cat rs/linear_rec.s


# <font color="green"> Problem 7 :  Tail recursive call</font>
* in it, define a function `linear_tail_rec` (or `Linear_tail_rec` in Go) that does the same thing as `linear_rec` in a slightly different way
* without details, you can understand the following C function computes the above $a_n$ if called with `linear_tail_rec(0, n, 1)`
* do the same for your language


In [ ]:
%%writefile cc/linear_tail_rec.c

long linear_tail_rec(long i, long n, long ai) {
  if (i == n) {
    return ai;
  } else {
    long aiplus1 = (34 * ai + 56) % 78;
    return linear_tail_rec(i + 1, n, aiplus1);
  }
}

In [ ]:
gcc -O3 -S cc/linear_tail_rec.c -o cc/linear_tail_rec.s
cat cc/linear_tail_rec.s


* generally, a function call whose return value becomes the return value of the caller is called a _tail call_
* for example, the function call `g(x)` below is a tail call, but `h(x)` is not
```
def f(x):
   if ...:
      return g(x)
   else:
      return h(x) + 1
```
* a _tail call_ that is also a recursive call is called a _tail recursive call_ and a compiler generally has a chance to optimize tail recursive calls
* a loop can generally be expressed as tail recursive functions
```
while A:
  B
```
can be translated into something like
```
def loop(...):
  if A:
     B
     loop(...)
```
* in particular, in OCaml, where loop syntax exist but more functional way of writing things is encouraged, writing what would a loop in other languages in tail recursive functions is a trick you want to master


* write the code below
* for Julia, do not forget to include `import InteractiveUtils` and `InteractiveUtils.code_native(YOUR_FUNCTION, (TYPE,...))` to show assembly code

In [ ]:
# replace xx with go, jl, ml, or rs and write the function
%%writefile xx/linear_tail_rec.c


* and see the assembly code 

In [ ]:
# uncomment commands for your language
# Go
# gccgo -O -g0 -S go/linear_tail_rec.go -o go/linear_tail_rec.s
# cat go/linear_tail_rec.go

# Julia
# julia jl/linear_tail_rec.jl

# OCaml
# ocamlopt -S ml/linear_tail_rec.ml
# cat ml/linear_tail_rec.ml

# Rust
# rustc -O --emit asm --crate-type lib rs/linear_tail_rec.rs -o rs/linear_tail_rec.s
# cat rs/linear_tail_rec.s


# <font color="green"> Problem 8 :  How arrays are scanned</font>
* in it, define a function `sum_vector_floats` (or `Sum_vector_floats` in Go) that takes an array of floating point numbers and returns the sum of all its elements (similar to to the following C function) in your language, compile it into assembly and examine it
* use
  * `[]float64` (slice) for Go
  * `Vector{Float64}` for Julia
  * `float array` (slice) for OCaml
  * `Vec<f64>` for Rust
* write them in
  * ordinary loop syntax (except OCaml)
  * tail-recursive functions

In [ ]:
%%writefile cc/sum_vector_floats.c

double sum_array_floats_loop(double * a, long n) {
  double s = 0.0;
  for (int i = 0; i < n; i++) {
    s += a[i];
  }
  return s;
}
double sum_array_floats_tail_rec(double * a, long i, long n, double s) {
  if (i == n) {
    return s;
  } else {
    return sum_array_floats_tail_rec(a, i + 1, n, s + a[i]);
  }
}

In [ ]:
gcc -O3 -S cc/sum_vector_floats.c -o cc/sum_vector_floats.s
cat cc/sum_vector_floats.s

* write the code below
* for Julia, do not forget to include `import InteractiveUtils` and `InteractiveUtils.code_native(YOUR_FUNCTION, (TYPE,...))` to show assembly code

In [ ]:
# replace xx with go, jl, ml, or rs and write the function
%%writefile xx/sum_vector_floats.c


* and see the assembly code 

In [ ]:
# uncomment commands for your language
# Go
# gccgo -O -g0 -S go/sum_vector_floats.go -o go/sum_vector_floats.s
# cat go/sum_vector_floats.go

# Julia
# julia jl/sum_vector_floats.jl

# OCaml
# ocamlopt -S ml/sum_vector_floats.ml
# cat ml/sum_vector_floats.ml

# Rust
# rustc -O --emit asm --crate-type lib rs/sum_vector_floats.rs -o rs/sum_vector_floats.s
# cat rs/sum_vector_floats.s

* __<font color="blue">Questions:</font>__
  * given that the array has a large number of elements, identify the loop that takes most of the time
  * in that loop, how many instructions are executed per array element?
  * will there be a performance difference between the loop version and tail recursive call version?
  * what does each language/compiler do to optimize it?
